In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
from decimal import Decimal, InvalidOperation, ROUND_HALF_UP
import re

class CalculatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Калькулятор")
        self.root.geometry("550x800")
        self.root.resizable(False, False)
        
        # Минимальное и максимальное значения
        self.MIN_VALUE = Decimal('-1000000000000.000000')
        self.MAX_VALUE = Decimal('1000000000000.000000')
        
        # Переменные для хранения данных
        self.num1_var = tk.StringVar()
        self.num2_var = tk.StringVar()
        self.result_var = tk.StringVar()
        self.operation_var = tk.StringVar(value="+")
        
        # Переменные для ошибок
        self.num1_error_var = tk.StringVar()
        self.num2_error_var = tk.StringVar()
        
        self.setup_ui()
        
    def setup_ui(self):
        # Заголовок
        title_label = tk.Label(
            self.root, 
            text="КАЛЬКУЛЯТОР", 
            font=("Arial", 20, "bold"),
            fg="blue"
        )
        title_label.pack(pady=10)
        
        # Информация о студенте
        info_frame = tk.Frame(self.root)
        info_frame.pack(pady=10)
        
        student_info = [
            "ФИО: Ломова Виктория Александровна",
            "4 курс",
            "4 группа", 
            "2025 год"
        ]
        
        for info in student_info:
            label = tk.Label(
                info_frame, 
                text=info,
                font=("Arial", 12),
                anchor="w"
            )
            label.pack(pady=2)
        
        # Основной фрейм для ввода
        main_frame = tk.Frame(self.root)
        main_frame.pack(pady=20, padx=20, fill="both")
        
        # Поле для первого числа
        tk.Label(main_frame, text="Первое число:", font=("Arial", 11)).grid(
            row=0, column=0, sticky="w", pady=5
        )
        
        num1_frame = tk.Frame(main_frame)
        num1_frame.grid(row=0, column=1, padx=10, pady=5, sticky="ew")
        
        self.num1_entry = tk.Entry(
            num1_frame, 
            textvariable=self.num1_var,
            font=("Arial", 12),
            width=25
        )
        self.num1_entry.pack(side="left", fill="x", expand=True)
        self.num1_entry.bind('<KeyRelease>', lambda e: self.validate_input(self.num1_entry, self.num1_error_var))
        self.num1_entry.bind('<Control-v>', lambda e: self.paste_clean(e, self.num1_var, self.num1_error_var))
        self.num1_entry.bind('<FocusOut>', lambda e: self.format_input_on_focusout(self.num1_entry, self.num1_var))
        
        # Сообщение об ошибке для первого числа
        self.num1_error_label = tk.Label(
            num1_frame,
            textvariable=self.num1_error_var,
            font=("Arial", 9),
            fg="red",
            wraplength=300
        )
        self.num1_error_label.pack(side="left", padx=5)
        
        # Поле для второго числа
        tk.Label(main_frame, text="Второе число:", font=("Arial", 11)).grid(
            row=1, column=0, sticky="w", pady=5
        )
        
        num2_frame = tk.Frame(main_frame)
        num2_frame.grid(row=1, column=1, padx=10, pady=5, sticky="ew")
        
        self.num2_entry = tk.Entry(
            num2_frame, 
            textvariable=self.num2_var,
            font=("Arial", 12),
            width=25
        )
        self.num2_entry.pack(side="left", fill="x", expand=True)
        self.num2_entry.bind('<KeyRelease>', lambda e: self.validate_input(self.num2_entry, self.num2_error_var))
        self.num2_entry.bind('<Control-v>', lambda e: self.paste_clean(e, self.num2_var, self.num2_error_var))
        self.num2_entry.bind('<FocusOut>', lambda e: self.format_input_on_focusout(self.num2_entry, self.num2_var))
        
        # Сообщение об ошибке для второго числа
        self.num2_error_label = tk.Label(
            num2_frame,
            textvariable=self.num2_error_var,
            font=("Arial", 9),
            fg="red",
            wraplength=300
        )
        self.num2_error_label.pack(side="left", padx=5)
        
        # Выбор операции
        tk.Label(main_frame, text="Операция:", font=("Arial", 11)).grid(
            row=2, column=0, sticky="w", pady=15
        )
        
        operation_frame = tk.Frame(main_frame)
        operation_frame.grid(row=2, column=1, padx=10, pady=15, sticky="w")
        
        operations = [
            ("Сложение (+)", "+"),
            ("Вычитание (-)", "-"),
            ("Умножение (×)", "×"),
            ("Деление (÷)", "÷")
        ]
        
        for i, (text, value) in enumerate(operations):
            tk.Radiobutton(
                operation_frame,
                text=text,
                variable=self.operation_var,
                value=value,
                font=("Arial", 11)
            ).grid(row=i//2, column=i%2, sticky="w", padx=10, pady=2)
        
        # Кнопки
        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=20)
        
        tk.Button(
            button_frame,
            text="Вычислить",
            command=self.calculate,
            font=("Arial", 12, "bold"),
            bg="lightgreen",
            padx=20,
            pady=10
        ).pack(side="left", padx=10)
        
        tk.Button(
            button_frame,
            text="Очистить",
            command=self.clear_all,
            font=("Arial", 12),
            bg="lightcoral",
            padx=20,
            pady=10
        ).pack(side="left", padx=10)
        
        # Результат
        result_frame = tk.Frame(self.root)
        result_frame.pack(pady=20, padx=20, fill="x")
        
        tk.Label(result_frame, text="Результат:", font=("Arial", 12, "bold")).pack(anchor="w")
        
        self.result_label = tk.Label(
            result_frame,
            textvariable=self.result_var,
            font=("Courier New", 14, "bold"),
            bg="lightyellow",
            relief="solid",
            width=35,
            height=2
        )
        self.result_label.pack(pady=10, fill="x")
        
        # Инструкция
        instruction_frame = tk.Frame(self.root)
        instruction_frame.pack(pady=10, padx=20)
        
        instructions = [
            "Инструкция:",
            "• Используйте Ctrl+C и Ctrl+V для копирования/вставки",
            "• Поддерживаются как точка, так и запятая",
            "• Можно вводить числа с пробелами или без них",
            "• При потере фокуса числа форматируются с пробелами",
            "• Диапазон: от -1 000 000 000 000.000000 до +1 000 000 000 000.000000",
            "• Числа остаются в полях после вычисления"
        ]
        
        for instruction in instructions:
            label = tk.Label(
                instruction_frame,
                text=instruction,
                font=("Arial", 9),
                anchor="w",
                justify="left"
            )
            label.pack(anchor="w")
    
    def check_spaces(self, text):
        """Проверка корректности пробелов-разделителей (если они есть)"""
        if not text:
            return True, ""
        
        # Если нет пробелов - всё ок
        if ' ' not in text:
            return True, ""
        
        # Проверяем на несколько пробелов подряд
        if '  ' in text:
            return False, "Несколько пробелов подряд"
        
        # Убираем знак минуса для проверки
        check_text = text.lstrip('-')
        
        # Проверяем, есть ли дробная часть
        if '.' in check_text or ',' in check_text:
            # Определяем разделитель
            separator = '.' if '.' in check_text else ','
            integer_part = check_text.split(separator)[0]
            fractional_part = check_text.split(separator)[1] if len(check_text.split(separator)) > 1 else ""
            
            # Проверяем пробелы в дробной части
            if ' ' in fractional_part:
                return False, "Пробелы в дробной части недопустимы"
        else:
            integer_part = check_text
            fractional_part = ""
        
        # Убираем все пробелы для проверки групп
        integer_without_spaces = integer_part.replace(' ', '')
        
        # Проверяем, что после удаления пробелов остаются только цифры
        if integer_without_spaces and not integer_without_spaces.isdigit():
            return False, "Недопустимые символы в целой части"
        
        return True, ""
    
    def validate_input(self, entry_widget, error_var):
        """Валидация ввода в реальном времени"""
        text = entry_widget.get()
        
        # Сбрасываем ошибку
        error_var.set("")
        entry_widget.config(bg="white")
        
        if not text:
            return
        
        # Проверяем на несколько знаков плюса
        if text.count('+') > 1:
            error_var.set("Некорректный ввод: несколько знаков +")
            entry_widget.config(bg="#FFE6E6")
            return
        
        # Проверяем на несколько точек/запятых
        dot_count = text.count('.') + text.count(',')
        if dot_count > 1:
            error_var.set("Некорректный ввод: несколько разделителей")
            entry_widget.config(bg="#FFE6E6")
            return
        
        # Проверяем положение минуса
        minus_positions = [i for i, char in enumerate(text) if char == '-']
        if minus_positions:
            # Если минус не в начале
            if minus_positions[0] != 0:
                error_var.set("Некорректный ввод: минус должен быть только в начале")
                entry_widget.config(bg="#FFE6E6")
                return
            # Если больше одного минуса
            if len(minus_positions) > 1:
                error_var.set("Некорректный ввод: несколько знаков -")
                entry_widget.config(bg="#FFE6E6")
                return
        
        # Проверяем минус после точки/запятой
        if '.' in text or ',' in text:
            separator = '.' if '.' in text else ','
            parts = text.split(separator)
            if len(parts) > 1 and '-' in parts[1]:
                error_var.set("Некорректный ввод: минус после разделителя")
                entry_widget.config(bg="#FFE6E6")
                return
        
        # Проверяем пробелы (если они есть)
        space_valid, space_error = self.check_spaces(text)
        if not space_valid:
            error_var.set(f"Некорректный ввод: {space_error}")
            entry_widget.config(bg="#FFE6E6")
            return
        
        # Заменяем запятую на точку
        text = text.replace(',', '.')
        
        # Проверяем на допустимые символы (теперь разрешаем цифры, минус, точку и пробелы)
        allowed_pattern = r'^[-]?[\d\s]*\.?[\d\s]*$'
        if not re.match(allowed_pattern, text):
            error_var.set("Некорректный ввод: недопустимые символы")
            entry_widget.config(bg="#FFE6E6")
            return
        
        # Удаляем все пробелы для дальнейшей обработки
        text_no_spaces = text.replace(' ', '')
        
        # Проверяем случаи типа ".123" или "-.123"
        if text_no_spaces.startswith('.') or text_no_spaces.startswith('-.'):
            # Это допустимо, преобразуем позже
            pass
        
        # Проверяем длину целой и дробной части
        if '.' in text_no_spaces:
            integer_part, fractional_part = text_no_spaces.split('.')
            # Убираем минус из integer_part для проверки длины
            integer_part_for_check = integer_part.lstrip('-')
            
            if integer_part_for_check and len(integer_part_for_check) > 13:
                error_var.set("Слишком длинная целая часть (макс. 13 цифр)")
                entry_widget.config(bg="#FFE6E6")
                return
            
            if len(fractional_part) > 6:
                error_var.set("Слишком длинная дробная часть (макс. 6 цифр)")
                entry_widget.config(bg="#FFE6E6")
                return
        else:
            integer_part_for_check = text_no_spaces.lstrip('-')
            if integer_part_for_check and len(integer_part_for_check) > 13:
                error_var.set("Слишком длинная целая часть (макс. 13 цифр)")
                entry_widget.config(bg="#FFE6E6")
                return
    
    def format_input_on_focusout(self, entry_widget, var):
        """Форматирование ввода при потере фокуса"""
        text = var.get()
        if not text:
            return
        
        # Удаляем все пробелы
        text_no_spaces = text.replace(' ', '')
        
        # Заменяем запятую на точку
        text_no_spaces = text_no_spaces.replace(',', '.')
        
        # Форматируем с пробелами для красивого отображения
        formatted_text = self.format_number_with_spaces(text_no_spaces)
        
        # Устанавливаем отформатированное значение
        var.set(formatted_text)
    
    def format_number_with_spaces(self, text):
        """Форматирование числа с пробелами-разделителями"""
        if not text:
            return ""
        
        # Проверяем знак
        sign = "-" if text.startswith('-') else ""
        text = text.lstrip('-')
        
        # Если текст начинается с точки
        if text.startswith('.'):
            return sign + text
        
        # Разделяем целую и дробную части
        if '.' in text:
            integer_part, fractional_part = text.split('.')
        else:
            integer_part, fractional_part = text, ""
        
        # Добавляем пробелы в целую часть
        if integer_part:
            # Форматируем целую часть с пробелами
            formatted_integer = ""
            for i, digit in enumerate(reversed(integer_part)):
                if i > 0 and i % 3 == 0:
                    formatted_integer = ' ' + formatted_integer
                formatted_integer = digit + formatted_integer
            
            result = sign + formatted_integer
        else:
            result = sign + "0"
        
        # Добавляем дробную часть
        if fractional_part:
            result += '.' + fractional_part
        
        return result
    
    def paste_clean(self, event, var, error_var):
        """Очистка вставляемого текста от недопустимых символов"""
        try:
            clipboard_text = self.root.clipboard_get()
        except:
            return
        
        # Удаляем все символы, кроме цифр, минуса, точки, запятой и пробелов
        cleaned = re.sub(r'[^\d\-\.,\s]', '', clipboard_text)
        
        # Заменяем запятые на точки
        cleaned = cleaned.replace(',', '.')
        
        # Удаляем лишние пробелы (оставляем только одиночные)
        cleaned = re.sub(r'\s+', ' ', cleaned).strip()
        
        # Проверяем на несколько минусов (оставляем только первый)
        if cleaned.count('-') > 1:
            cleaned = cleaned[0] + cleaned[1:].replace('-', '')
        
        # Проверяем на несколько точек
        if cleaned.count('.') > 1:
            parts = cleaned.split('.')
            cleaned = parts[0] + '.' + ''.join(parts[1:])
        
        # Устанавливаем очищенное значение
        var.set(cleaned)
        
        # Вызываем валидацию
        if var == self.num1_var:
            self.validate_input(self.num1_entry, self.num1_error_var)
        else:
            self.validate_input(self.num2_entry, self.num2_error_var)
        
        return "break"
    
    def parse_number(self, num_str):
        """Парсинг числа из строки"""
        if not num_str:
            return None, "Пустой ввод"
        
        # Удаляем все пробелы
        num_str = num_str.replace(' ', '')
        
        # Заменяем запятую на точку
        num_str = num_str.replace(',', '.')
        
        try:
            # Проверяем, что строка является валидным числом
            if not re.match(r'^-?\d*\.?\d*$', num_str):
                return None, "Некорректный формат числа"
            
            # Убираем лишние символы
            num_str = num_str.strip()
            
            # Проверяем случаи типа ".123" или "-.123"
            if num_str.startswith('.') or num_str.startswith('-.'):
                num_str = '0' + num_str if num_str.startswith('.') else '-0' + num_str[1:]
            
            # Проверяем пустую строку или только минус
            if not num_str or num_str == '-':
                return Decimal('0'), ""
            
            # Преобразуем в Decimal
            num = Decimal(num_str)
            
            # Проверяем диапазон
            if num < self.MIN_VALUE or num > self.MAX_VALUE:
                return None, f"Число вне диапазона ({self.MIN_VALUE} до {self.MAX_VALUE})"
            
            return num, ""
        except (InvalidOperation, ValueError) as e:
            return None, f"Ошибка преобразования: {str(e)}"
    
    def format_result(self, number):
        """Форматирование результата для отображения"""
        if number is None:
            return "Ошибка"
        
        # Преобразуем в Decimal если это не так
        if not isinstance(number, Decimal):
            try:
                number = Decimal(str(number))
            except:
                return "Ошибка формата"
        
        # Округляем до 6 знаков после запятой
        try:
            number = number.quantize(Decimal('0.000001'), rounding=ROUND_HALF_UP)
        except:
            pass
        
        # Преобразуем в строку
        num_str = format(number, 'f')
        
        # Разделяем на целую и дробную части
        if '.' in num_str:
            integer_part, fractional_part = num_str.split('.')
        else:
            integer_part, fractional_part = num_str, "000000"
        
        # Убираем знак минуса для форматирования
        sign = "-" if integer_part.startswith('-') else ""
        integer_part = integer_part.lstrip('-')
        
        # Если целая часть пустая (число между -1 и 1)
        if not integer_part:
            integer_part = "0"
        
        # Форматируем целую часть с пробелами
        formatted_integer = ""
        for i, digit in enumerate(reversed(integer_part)):
            if i > 0 and i % 3 == 0:
                formatted_integer = ' ' + formatted_integer
            formatted_integer = digit + formatted_integer
        
        # Обрабатываем дробную часть
        # Обеспечиваем ровно 6 цифр
        if len(fractional_part) < 6:
            fractional_part = fractional_part.ljust(6, '0')
        elif len(fractional_part) > 6:
            fractional_part = fractional_part[:6]
        
        # Убираем незначащие нули справа
        fractional_part = fractional_part.rstrip('0')
        
        # Формируем результат
        result = sign + formatted_integer
        if fractional_part:
            result += '.' + fractional_part
        
        return result
    
    def calculate(self):
        """Выполнение вычислений"""
        # Сбрасываем ошибки
        self.num1_error_var.set("")
        self.num2_error_var.set("")
        
        # Получаем числа
        num1_str = self.num1_var.get()
        num2_str = self.num2_var.get()
        
        # Проверяем, что оба числа введены
        if not num1_str:
            self.num1_error_var.set("Введите число")
            self.num1_entry.config(bg="#FFE6E6")
            return
        
        if not num2_str:
            self.num2_error_var.set("Введите число")
            self.num2_entry.config(bg="#FFE6E6")
            return
        
        # Парсим числа
        num1, error1 = self.parse_number(num1_str)
        num2, error2 = self.parse_number(num2_str)
        
        if error1:
            self.num1_error_var.set(error1)
            self.num1_entry.config(bg="#FFE6E6")
            return
        
        if error2:
            self.num2_error_var.set(error2)
            self.num2_entry.config(bg="#FFE6E6")
            return
        
        # Получаем операцию
        operation = self.operation_var.get()
        
        # Выполняем операцию
        try:
            if operation == "+":
                result = num1 + num2
            elif operation == "-":
                result = num1 - num2
            elif operation == "×":
                result = num1 * num2
            else:  # operation == "÷"
                if num2 == 0:
                    self.result_var.set("Ошибка: деление на ноль")
                    self.num2_error_var.set("Деление на ноль")
                    self.num2_entry.config(bg="#FFE6E6")
                    return
                # Деление с округлением до 6 знаков
                result = (num1 / num2).quantize(Decimal('0.000001'), rounding=ROUND_HALF_UP)
            
            # Проверяем результат на переполнение
            if result < self.MIN_VALUE or result > self.MAX_VALUE:
                self.result_var.set("ПЕРЕПОЛНЕНИЕ")
                return
            
            # Форматируем и отображаем результат
            result_str = self.format_result(result)
            self.result_var.set(result_str)
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка при вычислении: {str(e)}")
    
    def clear_all(self):
        """Очистка всех полей"""
        self.num1_var.set("")
        self.num2_var.set("")
        self.result_var.set("")
        self.operation_var.set("+")
        self.num1_error_var.set("")
        self.num2_error_var.set("")
        self.num1_entry.config(bg="white")
        self.num2_entry.config(bg="white")
        self.num1_entry.focus_set()

def main():
    root = tk.Tk()
    app = CalculatorApp(root)
    
    # Глобальные горячие клавиши (работают везде в приложении)
    def global_copy(event):
        widget = root.focus_get()
        if isinstance(widget, tk.Entry):
            if widget.selection_present():
                selected_text = widget.selection_get()
                root.clipboard_clear()
                root.clipboard_append(selected_text)
            return "break"
    
    def global_paste(event):
        widget = root.focus_get()
        if isinstance(widget, tk.Entry):
            try:
                clipboard_text = root.clipboard_get()
            except:
                return "break"
            
            # Очистка текста
            cleaned = re.sub(r'[^\d\-,.]', '', clipboard_text)
            cleaned = cleaned.replace(',', '.')
            
            if cleaned.count('-') > 1:
                cleaned = cleaned[0] + cleaned[1:].replace('-', '')
            
            if cleaned.count('.') > 1:
                parts = cleaned.split('.')
                cleaned = parts[0] + '.' + ''.join(parts[1:])
            
            # Вставка
            current_text = widget.get()
            if widget.selection_present():
                start = widget.index("sel.first")
                end = widget.index("sel.last")
                new_text = current_text[:start] + cleaned + current_text[end:]
                widget.delete(0, tk.END)
                widget.insert(0, new_text)
                widget.icursor(start + len(cleaned))
            else:
                cursor_pos = widget.index(tk.INSERT)
                new_text = current_text[:cursor_pos] + cleaned + current_text[cursor_pos:]
                widget.delete(0, tk.END)
                widget.insert(0, new_text)
                widget.icursor(cursor_pos + len(cleaned))
            
            # Вызываем валидацию
            if widget == app.num1_entry:
                app.validate_input(app.num1_entry)
            elif widget == app.num2_entry:
                app.validate_input(app.num2_entry)
            
            return "break"
    
    # Регистрируем глобальные обработчики по кодам клавиш
    # Это работает независимо от раскладки
    root.bind_all('<Control-Key-c>', global_copy)
    root.bind_all('<Control-Key-C>', global_copy)
    root.bind_all('<Control-Key-v>', global_paste)
    root.bind_all('<Control-Key-V>', global_paste)
    
    # Альтернативный вариант - проверка по коду клавиши
    root.bind_all('<Control-KeyPress>', lambda e: handle_ctrl_keypress(e, root, app))
    
    root.mainloop()

def handle_ctrl_keypress(event, root, app):
    """Обработка Ctrl+клавиша по коду"""
    if event.state & 0x4:  # Ctrl нажат
        keycode = event.keycode
        
        # Копирование: C (67) в любой раскладке
        if keycode == 67:
            widget = root.focus_get()
            if isinstance(widget, tk.Entry):
                if widget.selection_present():
                    selected_text = widget.selection_get()
                    root.clipboard_clear()
                    root.clipboard_append(selected_text)
                return "break"
        
        # Вставка: V (86) в любой раскладке
        elif keycode == 86:
            widget = root.focus_get()
            if isinstance(widget, tk.Entry):
                try:
                    clipboard_text = root.clipboard_get()
                except:
                    return "break"
                
                # Очистка текста
                cleaned = re.sub(r'[^\d\-,.]', '', clipboard_text)
                cleaned = cleaned.replace(',', '.')
                
                if cleaned.count('-') > 1:
                    cleaned = cleaned[0] + cleaned[1:].replace('-', '')
                
                if cleaned.count('.') > 1:
                    parts = cleaned.split('.')
                    cleaned = parts[0] + '.' + ''.join(parts[1:])
                
                # Вставка
                current_text = widget.get()
                if widget.selection_present():
                    start = widget.index("sel.first")
                    end = widget.index("sel.last")
                    new_text = current_text[:start] + cleaned + current_text[end:]
                    widget.delete(0, tk.END)
                    widget.insert(0, new_text)
                    widget.icursor(start + len(cleaned))
                else:
                    cursor_pos = widget.index(tk.INSERT)
                    new_text = current_text[:cursor_pos] + cleaned + current_text[cursor_pos:]
                    widget.delete(0, tk.END)
                    widget.insert(0, new_text)
                    widget.icursor(cursor_pos + len(cleaned))
                
                # Вызываем валидацию
                if widget == app.num1_entry:
                    app.validate_input(app.num1_entry)
                elif widget == app.num2_entry:
                    app.validate_input(app.num2_entry)
                
                return "break"


if __name__ == "__main__":
    main()